In [1]:
DEARGADAOL_LANDING = "https://www.leighleat.com/sc%C3%A9alta/45"

In [2]:
import requests

DEARGADAOL_LANDING_REQ = requests.get(DEARGADAOL_LANDING)

In [3]:
assert DEARGADAOL_LANDING_REQ.status_code == 200

In [4]:
from bs4 import BeautifulSoup


In [5]:
soup = BeautifulSoup(DEARGADAOL_LANDING_REQ.content, "html.parser")

In [12]:
pages = set()
for row in soup.find_all("div", {'class': 'row'}):
    for anchor in row.find_all("a"):
        if anchor["href"].startswith("/pages/"):
            pages.add(f'https://www.leighleat.com{anchor["href"]}')

In [13]:
pages

{'https://www.leighleat.com/pages/474',
 'https://www.leighleat.com/pages/475',
 'https://www.leighleat.com/pages/476',
 'https://www.leighleat.com/pages/477',
 'https://www.leighleat.com/pages/478',
 'https://www.leighleat.com/pages/479',
 'https://www.leighleat.com/pages/480',
 'https://www.leighleat.com/pages/481',
 'https://www.leighleat.com/pages/482',
 'https://www.leighleat.com/pages/483',
 'https://www.leighleat.com/pages/484',
 'https://www.leighleat.com/pages/485',
 'https://www.leighleat.com/pages/486',
 'https://www.leighleat.com/pages/520'}